In [10]:
from dotenv import load_dotenv, find_dotenv
import warnings
warnings.simplefilter('ignore')

In [11]:
load_dotenv(find_dotenv('file.env'))

True

In [12]:
import openai,langchain
print("langchain->",langchain.__version__, "openai->",openai.__version__)

langchain-> 0.2.8 openai-> 1.35.14


# Role  Prompting

Role prompting involves asking the LLM to assume a specific role or identity before performing a given task, such as acting as a copywriter. This can help guide the model's response by providing a context or perspective for the task. To work with role prompts, you could iteratively:

1- Specify the role in your prompt, e.g., "As a copywriter, create some attention-grabbing taglines for AWS services."

2 - Use the prompt to generate an output from an LLM.

3 - Analyze the generated response and, if necessary, refine the prompt for better results.

In [13]:
# In this example, the LLM is asked to act as a futuristic robot band 
# conductor and suggest a song title related to the given theme and year

from langchain import PromptTemplate, LLMChain
from langchain.llms import OpenAI

llm = OpenAI(model = "gpt-3.5-turbo-instruct", temperature=0)

template = """"
            As a futuristic robot band conductor, I need you to help me come up with a song title.
            What's a cool song title for a song about {theme} in the year {year}?
"""

prompt = PromptTemplate(
    input_variables = ["theme", "year"],
    template = template)

input_data = {"theme":"interstellar travel","year":"3030"}

chain = LLMChain(llm = llm, prompt=prompt)

response = chain.invoke(input_data)

print("Theme: interstellar travel")
print("Year: 3030")
print("AI-generated song title:", response['text'])

Theme: interstellar travel
Year: 3030
AI-generated song title: 
"Galactic Odyssey: Journey to 3030"


This is a good prompt for several reasons:

- **Clear instructions:** The prompt is phrased as a clear request for help in generating a song title, and it specifies the context: "As a futuristic robot band conductor." This helps the LLM understand that the desired output should be a song title related to a futuristic scenario.

- **Specificity:** The prompt asks for a song title that relates to a specific theme and a specific year, "{theme} in the year {year}." This provides enough context for the LLM to generate a relevant and creative output. The prompt can be adapted for different themes and years by using input variables, making it versatile and reusable.

- **Open-ended creativity:** The prompt allows for open-ended creativity, as it doesn't limit the LLM to a particular format or style for the song title. The LLM can generate a diverse range of song titles based on the given theme and year.
Focus on the task: The prompt is focused solely on generating a song title, making it easier for the LLM to provide a suitable output without getting sidetracked by unrelated topics.

These elements help the LLM understand the user's intention and generate a suitable response.

# Few Shot Prompting

In [14]:
# Few Shot Prompting In the next example, the LLM is asked to provide the emotion 
# associated with a given color based on a few examples of color-emotion pairs:
from langchain import FewShotPromptTemplate
examples = [
    {"color": "red", "emotion": "passion"},
    {"color": "blue", "emotion": "serenity"},
    {"color": "green", "emotion": "tranquility"},
]

example_formatter_template = """
Color: {color}
Emotion: {emotion}\n
"""

example_prompt = PromptTemplate(
    input_variables=["color", "emotion"],
    template=example_formatter_template,
)

few_shot_prompt = FewShotPromptTemplate(
    examples = examples,
    example_prompt = example_prompt,
    prefix = "Here are some examples of colors and the emotions associated with them:\n\n",
    suffix="\n\nNow, given a new color, identify the emotion associated with it:\n\nColor: {input}\nEmotion:",
    input_variables=  ["input"],
    example_separator = "\n")

formatted_prompt = few_shot_prompt.format(input="purple")

# The formatted prompt is not passed directly to the chain but type casted as a PromptTemplate object
chain = LLMChain(llm = llm, prompt=PromptTemplate(template=formatted_prompt))

response  = chain.invoke({})

print("Colour","purple")
print("Emotion",response['text'])

Colour purple
Emotion  royalty


# Bad Prompt practices

In [15]:
#  a too-vague prompt that provides little context or guidance for the model to generate a meaningful response.

template = "Tell me something about {topic}."
prompt = PromptTemplate(
    input_variables=["topic"],
    template=template,
)
prompt.format(topic="dogs")

'Tell me something about dogs.'

# Chain Prompting

Chain Prompting refers to the practice of chaining consecutive prompts, where the output of a previous prompt becomes the input of the successive prompt.

To use chain prompting with LangChain, you could:

- Extract relevant information from the generated response.
- Use the extracted information to create a new prompt that builds upon the previous response.
- Repeat steps as needed until the desired output is achieved.

**PromptTemplate** class makes constructing prompts with dynamic inputs easier. This is useful when creating a prompt chain that depends on previous answers. 

In [16]:
# prompt 1
template_question = """What is the anme of the famous scientist who developed the theory of general relativity?
Answer:"""

question_prompt = PromptTemplate(input_variabls=[], template = template_question)

# prompt 2

template_fact = """Provide a brief description of {scientist}'s thoery of general relativity.
Answer:
"""

fact_prompt = PromptTemplate(input_variables=['scientist'], template=template_fact)

# create llmchain for the first prompt

chain_question = LLMChain(llm=llm, prompt=question_prompt)

# running with empty input

response_question = chain_question.run({})

# extracting name from response
scientist = response_question.strip()

# create llmchain for the 2nd prompt

chain_fact = LLMChain(llm=llm, prompt = fact_prompt)

# Input data for the second prompt
input_data = {"scientist": scientist}

# Run the LLMChain for the second prompt
response_fact = chain_fact.run(input_data)

print("Scientist:", scientist)
print()
print("Fact:", response_fact)

Scientist: Albert Einstein

Fact: Albert Einstein's theory of general relativity is a theory of gravity that explains how massive objects interact with each other and how they affect the fabric of space and time. According to this theory, gravity is not a force between masses, but rather a curvature of space and time caused by the presence of massive objects. This curvature is what causes objects to move towards each other, giving the appearance of a gravitational force. General relativity also predicts the phenomenon of gravitational time dilation, where time moves slower in the presence of strong gravitational fields. This theory has been extensively tested and has been shown to accurately predict the behavior of objects in the universe, including the bending of light by massive objects and the existence of black holes. It is considered one of the most important and influential theories in modern physics.


**Bad Prompt Example**

This following fact_prompt may generate a less informative or focused response than the previous example due to its more open-ended nature.

fact_prompt = Tell me something interesting about {scientist}.

In [17]:
template_fact = """Tell me something interesting about {scientist}.
Answer:
"""

fact_prompt = PromptTemplate(input_variables=['scientist'], template=template_fact)

chain_fact = LLMChain(llm=llm, prompt = fact_prompt)

response_fact = chain_fact.run(input_data)

print("Scientist:", scientist)
print()
print("Fact:", response_fact)

Scientist: Albert Einstein

Fact: 
Albert Einstein was a passionate musician and played the violin. He often said that if he hadn't become a scientist, he would have become a musician. He even said that his theory of relativity was inspired by his love for music and the harmony of the universe.


**Unclear prompt example**

In [18]:
# Prompt 1
template_question = """What are some musical genres?
Answer: """
prompt_question = PromptTemplate(input_variables=[], template=template_question)

# Prompt 2
template_fact = """Tell me something about {genre1}, {genre2}, and {genre3} without giving any specific details.
Answer: """
prompt_fact = PromptTemplate(input_variables=["genre1", "genre2", "genre3"], template=template_fact)

# Create the LLMChain for the first prompt
chain_question = LLMChain(llm=llm, prompt=prompt_question)

# Run the LLMChain for the first prompt with an empty dictionary
response_question = chain_question.run({})

# Assign three hardcoded genres
genre1, genre2, genre3 = "jazz", "pop", "rock"

# Create the LLMChain for the second prompt
chain_fact = LLMChain(llm=llm, prompt=prompt_fact)

# Input data for the second prompt
input_data = {"genre1": genre1, "genre2": genre2, "genre3": genre3}

# Run the LLMChain for the second prompt
response_fact = chain_fact.run(input_data)

print("Genres:", genre1, genre2, genre3)
print("Fact:", response_fact)

Genres: jazz pop rock
Fact: 
Jazz, pop, and rock are three popular genres of music that have evolved over time and have a wide range of styles and influences. They all have distinct characteristics and elements that make them unique, such as improvisation in jazz, catchy melodies in pop, and heavy guitar riffs in rock. These genres have also influenced and been influenced by each other, leading to the creation of sub-genres and fusion styles. They have a strong presence in mainstream music and have been embraced by audiences of all ages and backgrounds.


In this example, the second prompt is constructed poorly. It asks to "tell me something about {genre1}, {genre2}, and {genre3} without giving any specific details." This prompt is unclear, as it asks for information about the genres but also states not to provide specific details. This makes it difficult for the LLM to generate a coherent and informative response. As a result, the LLM may provide a less informative or confusing answer.

The first prompt asks for "some musical genres" **without specifying any criteria or context**, and the second prompt asks why the given genres are "unique" **without providing any guidance** on what aspects of uniqueness to focus on, such as their historical origins, stylistic features, or cultural significance.

# Chain of Thought Prompting

Chain of Thought Prompting (CoT) is a technique developed to encourage large language models to explain their reasoning process, leading to more accurate results. By providing few-shot exemplars demonstrating the reasoning process, the LLM is guided to explain its reasoning when answering the prompt. This approach has been found effective in improving results on tasks like arithmetic, common sense, and symbolic reasoning.

Firstly, It can decompose complex tasks into smaller tasks making it easier to understand and solve the problem. This is particularly useful for calculations, logic, or multi-step reasoning tasks. Second, CoT can guide the model through related prompts, helping generate more coherent and contextually relevant outputs. This can lead to more accurate and useful responses in tasks that require a deep understanding of the problem or domain.

There are some limitations to consider when using CoT. One limitation is that it has been found to yield performance gains only when used with models of approximately 100 billion parameters or larger; smaller models tend to produce illogical chains of thought, which can lead to worse accuracy than standard prompting. Another limitation is that CoT may not be equally effective for all tasks. It has been shown to be most effective for tasks involving arithmetic, common sense, and symbolic reasoning. For other types of tasks, the benefits of using CoT might be less pronounced or even counterproductive.

For effetive prompt engineering

- Be specific with your prompt: Provide enough context and detail to guide the LLM toward the desired output.
- Force conciseness when needed.
- Encourage the model to explain its reasoning: This can lead to more accurate results, especially for complex tasks.

In [33]:
examples = [
    {
        "query": "What's the secret to happiness?",
        "answer": "Finding balance in life and learning to enjoy the small moments."
    }, {
        "query": "How can I become more productive?",
        "answer": "Try prioritizing tasks, setting goals, and maintaining a healthy work-life balance."
    }
]

example_template = """
User: {query}
AI: {answer}
"""

example_prompt = PromptTemplate(
    input_variables=["query", "answer"],
    template=example_template
)

prefix = """The following are excerpts from conversations with an AI
life coach. The assistant provides insightful and practical advice to the users' questions. Here are some
examples: 
"""

suffix = """
User: {query}
AI: """

few_shot_prompt_template = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    prefix=prefix,
    suffix=suffix,
    input_variables=["query"],
    example_separator="\n\n"
)

user_query = "What are some tips for improving communication skills?"

formatted_template = few_shot_prompt_template.format(query = user_query)

chain = LLMChain(llm=llm, prompt=PromptTemplate(template = formatted_template))

response = chain.invoke({})

print("User Query:", user_query)
print("AI Response:", response)

response_run = chain.run({})

print(response_run)

User Query: What are some tips for improving communication skills?
AI Response: {'text': "Active listening, being clear and concise, and practicing empathy are all important skills for effective communication. Also, don't be afraid to ask for clarification or feedback to improve your communication."}
Active listening, being clear and concise, and practicing empathy are all important skills for effective communication. Also, don't be afraid to ask for clarification or feedback to improve your communication.


In [29]:
# creating chain withut formatted prompt, rather passing few shot prompt template directly
chain = LLMChain(llm=llm, prompt=few_shot_prompt_template)

response = chain.run({"query": user_query})

print("User Query:", user_query)
print("AI Response:", response)

User Query: What are some tips for improving communication skills?
AI Response: Active listening, being clear and concise, and practicing empathy are all important skills for effective communication. Also, don't be afraid to ask for clarification or feedback to improve your communication.


This prompt:

- **Provides a clear context in the prefix:** The prompt states that the AI is a life coach providing insightful and practical advice. This context helps guide the AI's responses and ensures they align with the intended purpose.

- **Uses examples** that demonstrate the AI's role and **the type of responses** it generates: By providing relevant examples, the AI can better understand the style and tone of the responses it should produce. These examples serve as a reference for the AI to generate similar responses that are consistent with the given context.

- Separates examples and the actual query: This **allows the AI to understand the format it should follow,** ensuring a clear distinction between example conversations and the user's input. This separation helps the AI to focus on the current query and respond accordingly.

- Includes a clear suffix that indicates where the user's input goes and where the AI should provide its response: The suffix acts as a cue for the AI, showing where the user's query ends and the AI's response should begin. This structure helps maintain a **clear and consistent format** for the generated responses.


By using this well-structured prompt, the AI can understand its role, the context, and the expected response format, leading to more accurate and useful outputs.